En este notebook se realiza la separación entre train y test, partiendo del data frame final. Se visualiza como estan distribuidos los valores del target en cada caso.

Además, se descartan columnas que tienen listado de tags, y aquella que tiene todas las categorías involucradas desde la 'root' hasta la más específica de la publicación. Las había dejado para probar codificar esa información con embeddings.

In [1]:
import json
from urllib.request import urlopen
import urllib
import pandas as pd
from tqdm.notebook import tqdm
import os
import matplotlib.pyplot as plt
import seaborn as sn

In [2]:
os.chdir("..") 

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.prompt { font-size: 08px !important; }</style>"))
pd.options.display.max_columns = None

In [4]:
seed = 1111

In [5]:
data_path = "data/"
input_path = data_path + "input/"

In [6]:
df = pd.read_csv(input_path + 'data.csv', sep='\t', encoding='utf-8', )  
df.head()

,installments.quantity,installments.rate,id,site_id,title,price,currency_id,available_quantity,sold_quantity,buying_mode,listing_type_id,condition,accepts_mercadopago,category_id,is_official_store,catalog_product_id,tags,catalog_listing,seller.id,seller.car_dealer,seller.real_estate_agency,seller.tags,seller.seller_reputation.transactions.total,seller.seller_reputation.transactions.canceled,seller.seller_reputation.transactions.period,seller.seller_reputation.transactions.ratings.negative,seller.seller_reputation.transactions.ratings.positive,seller.seller_reputation.transactions.ratings.neutral,seller.seller_reputation.transactions.completed,seller.seller_reputation.power_seller_status,seller.seller_reputation.metrics.claims.rate,seller.seller_reputation.metrics.delayed_handling_time.rate,seller.seller_reputation.metrics.sales.period,seller.seller_reputation.metrics.sales.completed,seller.seller_reputation.metrics.cancellations.rate,address.state_name,shipping.free_shipping,shipping.mode,shipping.tags,shipping.logistic_type,shipping.store_pick_up,seller.seller_reputation.real_level,root_category_id,root_category_name,discount,category_name,categories_from_root
0,12.0,70.17,MLA901948080,MLA,Estéreo Para Auto Pioneer Mvh S215bt Con Usb Y...,12600.0,ARS,1,5,buy_it_now,gold_special,new,True,MLA6867,False,MLA14983192,"['brand_verified', 'extended_warranty_eligible...",True,8911722,False,False,"['normal', 'user_info_verified', 'eshop', 'cre...",415.0,30.0,historic,0.01,0.99,0.00,385.0,silver,0.0000,0.0000,60 days,68.0,0.0000,Buenos Aires,True,me2,"['self_service_in', 'mandatory_free_shipping']",drop_off,False,NaN,MLA5725,Accesorios para Vehículos,NaN,Estéreos,"['Accesorios para Vehículos', 'Audio para Vehí..."
1,12.0,70.17,MLA869356478,MLA,Neumático Pirelli Formula Energy 185/60 R14 82h,10250.0,ARS,50,500,buy_it_now,gold_special,new,True,MLA22195,True,MLA15000133,"['brand_verified', 'good_quality_picture', 'go...",True,116026826,False,False,"['brand', 'user_info_verified', 'large_seller'...",22539.0,975.0,historic,0.01,0.96,0.03,21564.0,platinum,0.0039,0.0559,60 days,3662.0,0.0046,Buenos Aires,True,me2,['mandatory_free_shipping'],cross_docking,True,NaN,MLA5725,Accesorios para Vehículos,NaN,Neumáticos de Auto y Camioneta,"['Accesorios para Vehículos', 'Neumáticos', 0 ..."
2,12.0,70.17,MLA851109741,MLA,Carcasa Llave Navaja Peugeot 2 Bot 307 308 C/...,799.0,ARS,50,500,buy_it_now,gold_special,new,True,MLA373345,False,NaN,"['good_quality_picture', 'good_quality_thumbna...",NaN,435859328,False,False,"['normal', 'user_info_verified', 'eshop', 'msh...",7371.0,386.0,historic,0.01,0.98,0.01,6985.0,platinum,0.0058,0.0033,60 days,2313.0,0.0000,Santa Fe,False,me2,['fulfillment'],fulfillment,False,NaN,MLA5725,Accesorios para Vehículos,NaN,Llaves Telecomando,"['Accesorios para Vehículos', 'Repuestos Autos..."
3,9.0,0.00,MLA836983125,MLA,Funda Cubre Rueda Auxilio Ford Ecosport,891.0,ARS,100,250,buy_it_now,gold_pro,new,True,MLA403345,False,NaN,"['brand_verified', 'good_quality_picture', 'go...",NaN,20861971,False,False,"['normal', 'user_info_verified', 'credits_prof...",1036.0,31.0,historic,0.01,0.99,0.00,1005.0,gold,0.0000,0.0000,60 days,455.0,0.0000,Capital Federal,False,me2,"['fulfillment', 'self_service_in']",fulfillment,False,NaN,MLA5725,Accesorios para Vehículos,0.1,Fundas Cubre Ruedas,"['Accesorios para Vehículos', 'Accesorios de A..."
4,12.0,70.17,MLA899116906,MLA,Servicio De Envío Express De Telepase Mercado ...,655.0,ARS,500,5000,buy_it_now,gold_special,new,True,MLA455216,True,NaN,"['brand_verified', 'good_quality_picture', 'go...",NaN,679055037,False,False,"['brand', 'user_info_verified', 'large_seller'...",25361.0,1525.0,historic,0.03,0.96,0.01,23836.0,NaN,0.0017,0.0000,60 days,12476.0,0.0000,Capital Federal,True,me2,"['fulfillment', 'self_service_in']",fulfillment,False,NaN,MLA5725,Accesorios para Vehículos,NaN,Dispositivos para TelePASE,"['Accesorios para Vehículos', 0 ..."


In [7]:
df.columns.tolist()

['installments.quantity',
 'installments.rate',
 'id',
 'site_id',
 'title',
 'price',
 'currency_id',
 'available_quantity',
 'sold_quantity',
 'buying_mode',
 'listing_type_id',
 'condition',
 'accepts_mercadopago',
 'category_id',
 'is_official_store',
 'catalog_product_id',
 'tags',
 'catalog_listing',
 'seller.id',
 'seller.car_dealer',
 'seller.real_estate_agency',
 'seller.tags',
 'seller.seller_reputation.transactions.total',
 'seller.seller_reputation.transactions.canceled',
 'seller.seller_reputation.transactions.period',
 'seller.seller_reputation.transactions.ratings.negative',
 'seller.seller_reputation.transactions.ratings.positive',
 'seller.seller_reputation.transactions.ratings.neutral',
 'seller.seller_reputation.transactions.completed',
 'seller.seller_reputation.power_seller_status',
 'seller.seller_reputation.metrics.claims.rate',
 'seller.seller_reputation.metrics.delayed_handling_time.rate',
 'seller.seller_reputation.metrics.sales.period',
 'seller.seller_reputa

In [8]:
columns_discard = [
    'tags',
    'seller.tags', 
    'shipping.tags',
    'categories_from_root', 
    ]

In [9]:
df = df.drop(columns_discard, axis=1)

## Train vs test

In [10]:
train_df = df.sample(frac=0.75, random_state=seed)
train_df.shape


(23180, 43)

In [11]:
file_name = input_path + "train.csv"
train_df.to_csv(file_name, sep='\t', encoding='utf-8', index=False)

In [12]:
train_df.sold_quantity.value_counts().sort_index()

0        2325
1         212
2         187
3         173
4         161
5        1740
25       1379
50       2105
100      1599
150      1291
200      1086
250      3541
500      6859
5000      514
50000       8
Name: sold_quantity, dtype: int64

In [13]:
test_df = df.loc[~df.index.isin(train_df.index)]
test_df.shape

(7727, 43)

In [14]:
file_name = input_path + "test.csv"
test_df.to_csv(file_name, sep='\t', encoding='utf-8', index=False)

In [15]:
test_df.sold_quantity.value_counts().sort_index()

0         828
1          78
2          62
3          57
4          44
5         589
25        456
50        694
100       524
150       431
200       357
250      1143
500      2269
5000      193
50000       2
Name: sold_quantity, dtype: int64